### 5.4.1. 不带参数的层
构造一个没有任何参数的自定义层。下面的CenteredLayer类要从其输入中减去均值。
要构建他，我们只需要继承基础类并实现前行传播功能。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [6]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super(CenteredLayer, self).__init__()
    def forward(self, X):
        return X - X.mean()

In [7]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在，我们可以将层作为组件合并到更复杂的模型中。

In [10]:
net = nn.Sequential(nn.Linear(8,128), CenteredLayer())
Y = net(torch.rand(4,8))
Y.mean()

tensor(3.7253e-09, grad_fn=<MeanBackward0>)

### 5.4.2 带参数的层
以上我们知道了如何定义简单的层，下面我们继续定义具有参数的层， 这些参数可以通过训练进行调整。
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。
比如：管理访问，初始化，共享，保存和加载参数。

现在，让我们实现自定义版本的全连接层。 回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。 在此实现中，我们使用修正线性单元作为激活函数。 该层需要输入参数：in_units和units，分别表示输入数和输出数。

In [11]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super(MyLinear, self).__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units, ))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data)  + self.bias.data
        return F.relu(linear)

In [12]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.2245,  1.5342,  0.2318],
        [-0.4079,  0.5728,  0.1492],
        [-0.9342, -0.2301, -2.1522],
        [ 0.4209,  0.1665, -1.1555],
        [ 0.3044,  1.2553,  0.3923]], requires_grad=True)

In [13]:
linear(torch.rand(2, 5))

tensor([[0.0000, 1.9205, 0.0000],
        [0.0000, 1.2098, 0.0000]])

In [14]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])